# Estimating the true correlations between activity patterns
This jupyter notebook contains the simulations underlying the blog [How can we estimate the *true* correlation between two activity patterns?](http://diedrichsenlab.org/BrainDataScience/noisy_correlation/)

In [26]:
import numpy as np 
import PcmPy as pcm
from PcmPy import sim
import pandas as pd
from correlation_demo import plot_Figure2, plot_Figure3, plot_Figure4

## Simply correlating the mean activity patterns
First look at the problem of simply using the correlation between two measured activty patterns. First, we write a little function to calculate the correlation between two mean activity patterns. 

In [27]:
def get_corr(X,cond_vec):
    """
        Get normal correlation
    """
    p1 = X[cond_vec==0,:].mean(axis=0)
    p2 = X[cond_vec==1,:].mean(axis=0)
    return np.corrcoef(p1,p2)[0,1]

Now we use the pcm-toolbox to simulate data from a trye correlation model (M)

In [28]:
def do_sim(corr,signal=np.linspace(0,5,20),n_sim=50,randseed=None):
    M = pcm.CorrelationModel('corr',num_items=1,corr=corr,cond_effect=False)
    G,dG = M.predict([0,0])
    cond_vec,part_vec = pcm.sim.make_design(2,2)
    Lcorr = []
    Lsign = []
    rng = np.random.default_rng(randseed)

    for s in signal:
        D = pcm.sim.make_dataset(M, [0,0], cond_vec, n_sim=n_sim, signal=s,rng=rng)
        for i in range(n_sim):
            data = D[i].measurements
            Lcorr.append(get_corr(data,cond_vec))
            Lsign.append(s)

    S = pd.DataFrame({'r_naive':Lcorr,'signal':Lsign})
    S['true'] = np.ones((S.shape[0],))*corr
    return S

Now we can run the simulation for Figure 2. 

In [35]:
# Make a spacing of different signals
sig = np.linspace(0.1,5.1,21)# np.logspace(np.log(0.2),np.log(5),10)
# Get the simulations
D = do_sim(0.7,n_sim=200,signal=sig,randseed=10)
# Summarize the mean and std deviation
T = D.groupby("signal").apply(np.mean)
Tstd = D.groupby("signal").apply(np.std)
# Plot and show the Figure
fig = plot_Figure2(D,T,Tstd)
fig.update_layout(width=700,height=400)
fig.show()


This is a function that runs a specific simulation across a range of signal to noise levels. 

## Noise ceiling correction and cross-block estimates
We can calculate the noise ceiling to correct for the bias in the correlation. Here a function that uses the cross-measurement correlations to obtain an estimate of the noise ceiling. 

In [30]:
def get_noiseceil(X,cond_vec):
    """
        Calculate noise ceiling over reliabilities
    """
    rel = np.array([0.0,0.0])
    for i in [0,1]:
        N = np.sum(cond_vec==i) # Number of measurements
        R = np.corrcoef(X[cond_vec==i,:]) # Correlation matrix
        index_R = np.where(~np.eye(N,dtype=bool)) # Average cross-block correlations
        r = np.mean(R[index_R]) # Average the non-diagnal elements.
        rel[i] = r * N / (r*(N-1)+1) # Overall realibility of the mean

    # Check if both are above zero
    if rel[0]>0 and rel[1]>0:
        noise_r = np.sqrt(rel[0]*rel[1])
    else:
        noise_r = np.nan
    return noise_r


Alternatively we can use the cross-block information direction to caluculate correct coefficients. Here we use `est_G_crossval` to calculate those covariances. 

In [31]:
def get_crossblock(X,cond_vec,part_vec):
    """
        calculate the cross-block correlation
    """
    G = pcm.util.est_G_crossval(X,cond_vec,part_vec)
    var = G[0][0,0]*G[0][1,1]
    if var<=0:
        crosscorr = np.nan
    else:
        crosscorr = G[0][0,1]/np.sqrt(var)
    return crosscorr

Now we need to extend our do_sim function to get these addtional measures 

In [32]:
def do_sim(corr,signal=np.linspace(0,5,20),n_sim=50,randseed=None):
    M = pcm.CorrelationModel('corr',num_items=1,corr=corr,cond_effect=False)
    G,dG = M.predict([0,0])
    cond_vec,part_vec = pcm.sim.make_design(2,2)
    Lcorr = []
    LnoiseCeil = []
    Lsign = []
    LcrossBlock =[]
    rng = np.random.default_rng(randseed)

    for s in signal:
        D = pcm.sim.make_dataset(M, [0,0], cond_vec, n_sim=n_sim, signal=s,rng=rng)
        for i in range(n_sim):
            data = D[i].measurements
            Lcorr.append(get_corr(data,cond_vec))
            LnoiseCeil.append(get_noiseceil(data,cond_vec))
            LcrossBlock.append(get_crossblock(data,cond_vec,part_vec))
            Lsign.append(s)
    S = pd.DataFrame({'r_naive':Lcorr,'signal':Lsign,
                        'noiseCeil':LnoiseCeil,'cross_block':LcrossBlock})
    S['true'] = np.ones((S.shape[0],))*corr
    return S

Now we can do the simulation - if you want more reliable results the number of simulations should be increase to 10000. 

In [36]:
sig = np.array([0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.5,0.6,0.7,0.8,0.9,1.5,2,2.5,3])# np.logspace(np.log(0.2),np.log(5),10)
D = do_sim(0.7,n_sim=200,signal=sig,randseed=12)
D['noiseceil_nan'] = np.isnan(D.noiseCeil)
D['corr_corrected'] = D.r_naive / D.noiseCeil # Correction for noise ceiling 
D['corr_corrected_imp'] = D.corr_corrected
D.loc[np.isnan(D.corr_corrected),'corr_corrected_imp']=0
D['cross_block_imp'] = D.cross_block
D.loc[np.isnan(D.cross_block),'cross_block_imp']=0
T = D.groupby("signal").apply(np.mean)
Tstd = D.groupby("signal").apply(np.std)
fig = plot_Figure3(D,T,Tstd)
fig.update_layout(width=700,height=400)
fig.show()

## Solution 2: Using the likelihood
First we define a function that simulates data and then fits it with a series of PCM models. 

In [ ]:
def do_sim_corrpcm(corr=0.7,signal=0.5,n_sim=20,nsteps = 11,randseed=None):
    # Make the design in this case it's 2 runs, 2 conditions!
    cond_vec,part_vec = pcm.sim.make_design(2,2)
    # Generate different models from 0 to 1
    M=[]
    for r in np.linspace(0,1,nsteps):
        M.append(pcm.CorrelationModel(f"{r:0.2f}",num_items=1,corr=r,cond_effect=False))
    Mflex = pcm.CorrelationModel("flex",num_items=1,corr=None,cond_effect=False)
    M.append(Mflex)
    # Now do the simulations and fit with the models
    rng = np.random.default_rng(randseed)
    Mtrue = pcm.CorrelationModel('corr',num_items=1,corr=corr,cond_effect=False)
    D = pcm.sim.make_dataset(Mtrue, [0,0], cond_vec,part_vec=part_vec,n_sim=n_sim, signal=signal, rng=rng)
    T,theta = pcm.inference.fit_model_individ(D,M,fixed_effect=None,fit_scale=False,verbose = False)
    return T,theta,M

In [37]:
T,theta,M = do_sim_corrpcm(corr=0.7,signal=0.5,n_sim=20,nsteps=21,randseed=11)
fig=plot_Figure4(T,theta,M)
fig.update_layout(width=700,height=400)
fig.show()

/Users/jdiedrichsen/Python/PcmPy/PcmPy/model.py:261: RuntimeWarning:

overflow encountered in exp

/Users/jdiedrichsen/Python/PcmPy/PcmPy/model.py:261: RuntimeWarning:

invalid value encountered in true_divide



Note that the log-likelihood values are quite negative - and differ substantially across data sets. This is normal - the only thing we can interpret are differences between log-likelihoods across the same data set for different models. 

In [ ]:
T

variable likelihood                                                         \
model          0.00       0.05       0.10       0.15       0.20       0.25   
0        -80.213258 -79.975602 -79.743281 -79.518565 -79.302870 -79.097164   
1        -80.890808 -80.451681 -80.034392 -79.638146 -79.262343 -78.906596   
2        -73.952510 -73.814902 -73.679411 -73.547906 -73.421654 -73.301560   
3        -77.282291 -77.009676 -76.746336 -76.492943 -76.249959 -76.017729   
4        -83.491441 -83.240729 -82.997738 -82.763341 -82.538145 -82.322608   
5        -75.166672 -74.999432 -74.850917 -74.721235 -74.610569 -74.519197   
6        -68.803865 -68.285783 -67.788955 -67.312309 -66.854954 -66.416196   
7        -84.549192 -84.274535 -84.006158 -83.745562 -83.493709 -83.251258   
8        -64.024354 -63.773315 -63.537101 -63.316254 -63.111175 -62.922231   
9        -82.259471 -81.968971 -81.695224 -81.438186 -81.197853 -80.974303   
10       -69.506350 -69.254634 -68.993507 -68.727382 -68.459022 -68.190248   
11       -76.422573 -76.265195 -76.116268 -75.976486 -75.846375 -75.726349   
12       -82.964747 -82.891094 -82.820337 -82.753333 -82.690725 -82.632986   
13       -71.393642 -71.100513 -70.822960 -70.561127 -70.315129 -70.085116   
14       -76.469758 -76.166427 -75.888594 -75.635867 -75.408127 -75.205544   
15       -81.981675 -81.750162 -81.546200 -81.369590 -81.220451 -81.099240   
16       -88.695312 -88.212270 -87.744621 -87.291808 -86.853266 -86.428470   
17       -71.802322 -71.625012 -71.443005 -71.259127 -71.075346 -70.893079   
18       -62.851722 -62.766007 -62.687989 -62.619045 -62.559989 -62.511259   
19       -59.295262 -58.983127 -58.696915 -58.436189 -58.200798 -57.990900   

variable                                              ... iterations        \
model          0.30       0.35       0.40       0.45  ...       0.60  0.65   
0        -78.902173 -78.718499 -78.546692 -78.387295  ...        4.0   4.0   
1        -78.570757 -78.254942 -77.959566 -77.685386  ...        4.0   4.0   
2        -73.188299 -73.082393 -72.984258 -72.894235  ...        4.0   4.0   
3        -75.796557 -75.586744 -75.388627 -75.202605  ...        5.0   5.0   
4        -82.117107 -81.921991 -81.737618 -81.564381  ...        4.0   4.0   
5        -74.447497 -74.395945 -74.365090 -74.355498  ...        4.0   5.0   
6        -65.995559 -65.592813 -65.207999 -64.841480  ...        4.0   4.0   
7        -83.018714 -82.796504 -82.585041 -82.384759  ...        4.0   4.0   
8        -62.749818 -62.594393 -62.456499 -62.336772  ...        4.0   4.0   
9        -80.767729 -80.578462 -80.406992 -80.253985  ...        4.0   4.0   
10       -67.922293 -67.656023 -67.392023 -67.130751  ...        5.0   5.0   
11       -75.616760 -75.517921 -75.430114 -75.353597  ...        3.0   3.0   
12       -82.580462 -82.533389 -82.491909 -82.456077  ...        4.0   4.0   
13       -69.871314 -69.674063 -69.493837 -69.331266  ...        4.0   4.0   
14       -75.028607 -74.878147 -74.755371 -74.661893  ...        4.0   4.0   
15       -81.006775 -80.944257 -80.913298 -80.915935  ...        4.0   4.0   
16       -86.016980 -85.618459 -85.232726 -84.859761  ...        5.0   5.0   
17       -70.713379 -70.537050 -70.364734 -70.196961  ...       11.0  10.0   
18       -62.472989 -62.445029 -62.426923 -62.417874  ...        4.0   4.0   
19       -57.806979 -57.649878 -57.520837 -57.421526  ...        4.0   4.0   

variable                                              
model     0.70  0.75  0.80  0.85 0.90 0.95 1.00 flex  
0          4.0   4.0   4.0   4.0  4.0  4.0  4.0  7.0  
1          4.0   4.0   4.0   4.0  4.0  4.0  4.0  5.0  
2          4.0   4.0   4.0   4.0  4.0  4.0  3.0  6.0  
3          4.0   4.0   4.0   4.0  4.0  4.0  4.0  8.0  
4          4.0   4.0   4.0   4.0  4.0  4.0  4.0  8.0  
5          5.0   5.0   5.0   6.0  6.0  6.0  6.0  4.0  
6          4.0   4.0   4.0   4.0  4.0  4.0  4.0  7.0  
7          4.0   4.0   4.0   4.0  4.0  4.0  4.0  8.0  
8          4.0  